In [1]:
!pip install -q gradio transformers accelerate

In [2]:
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load your model
model_name = "medachbab/phi2-sql-v3"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

In [3]:
# SQL generation function
'''def generate_sql(context, question):
    prompt = f"""### Instruction:
Generate SQL query based on the following context and question.

### Context:
{context}

### Question:
{question}

### Response:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("### Response:")[-1].strip()'''
def generate_sql(context, question):
    prompt = f"""### Instruction:
Generate SQL query based on the following context and question.

### Context:
{context}

### Question:
{question}

### Response:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [1]:
import gradio as gr

tables = []

def save_table(name, n1, n2, n3, n4, n5, t1, t2, t3, t4, t5):
    if not name.strip():
        return "❌ Table name is required."

    cols = []
    for n, t in zip([n1, n2, n3, n4, n5], [t1, t2, t3, t4, t5]):
        if n.strip() and t:
            cols.append({"name": n.strip(), "type": t})

    if not cols:
        return "❌ Add at least one valid column."

    tables.append({
        "name": name.strip(),
        "columns": cols
    })

    return f"✅ Table '{name}' saved with {len(cols)} columns."

def generate_sql_context():
    if not tables:
        return "⚠️ No tables defined."

    stmts = []
    for table in tables:
        name = table["name"]
        cols = [f"{col['name']} {col['type']}" for col in table["columns"]]
        stmts.append(f"CREATE TABLE {name} (\n  " + ",\n  ".join(cols) + "\n);")

    return "\n\n".join(stmts)

def generate_sql(context, question):
    prompt = f"""### Instruction:
Generate SQL query based on the following context and question.

### Context:
{context}

### Question:
{question}

### Response:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("### Response:")[-1].strip()

def clear_all():
    tables.clear()
    return "", "", "", "", "", "", None, None, None, None, None, "", "", ""

with gr.Blocks(theme="default") as app:
    gr.Markdown("""
    # 🧠 SQL Table & Query Generator
    Easily define SQL tables, generate schemas, and ask questions to get SQL queries.
    """)

    with gr.Accordion("📋 Define Table Schema", open=True):
        with gr.Row():
            table_name = gr.Textbox(label="Table Name", placeholder="e.g. users")

        with gr.Row():
            col_name1 = gr.Textbox(label="Column 1 Name")
            col_type1 = gr.Dropdown(["INT", "VARCHAR", "TEXT", "DATE", "BOOLEAN", "FLOAT"], label="Type")

        with gr.Row():
            col_name2 = gr.Textbox(label="Column 2 Name")
            col_type2 = gr.Dropdown(["INT", "VARCHAR", "TEXT", "DATE", "BOOLEAN", "FLOAT"], label="Type")

        with gr.Row():
            col_name3 = gr.Textbox(label="Column 3 Name")
            col_type3 = gr.Dropdown(["INT", "VARCHAR", "TEXT", "DATE", "BOOLEAN", "FLOAT"], label="Type")

        with gr.Row():
            col_name4 = gr.Textbox(label="Column 4 Name (optional)")
            col_type4 = gr.Dropdown(["INT", "VARCHAR", "TEXT", "DATE", "BOOLEAN", "FLOAT"], label="Type")

        with gr.Row():
            col_name5 = gr.Textbox(label="Column 5 Name (optional)")
            col_type5 = gr.Dropdown(["INT", "VARCHAR", "TEXT", "DATE", "BOOLEAN", "FLOAT"], label="Type")

        with gr.Row():
            save_btn = gr.Button("💾 Save Table", variant="primary")
            clear_btn = gr.Button("🧹 Clear All", variant="secondary")
        status = gr.Textbox(label="Status", interactive=False)

    with gr.Accordion("🛠️ Generate SQL Schema", open=True):
        generate_sql_btn = gr.Button("⚙️ Generate SQL Context", variant="secondary")
        context_output = gr.Code(label="SQL Context", language="sql", interactive=False)

    with gr.Accordion("🔎 Ask SQL Question", open=True):
        question_input = gr.Textbox(label="Question", placeholder="e.g. Show all users older than 30", lines=2)
        ask_sql_btn = gr.Button("🚀 Generate SQL Query", variant="primary")
        final_sql_output = gr.Code(label="Generated SQL", language="sql", interactive=False)

    save_btn.click(
        save_table,
        inputs=[table_name, col_name1, col_name2, col_name3, col_name4, col_name5,
                col_type1, col_type2, col_type3, col_type4, col_type5],
        outputs=[status]
    )

    generate_sql_btn.click(
        generate_sql_context,
        outputs=[context_output]
    )

    ask_sql_btn.click(
        generate_sql,
        inputs=[context_output, question_input],
        outputs=[final_sql_output]
    )

    clear_btn.click(
        clear_all,
        outputs=[
            table_name, col_name1, col_name2, col_name3, col_name4, col_name5,
            col_type1, col_type2, col_type3, col_type4, col_type5,
            status, context_output, final_sql_output
        ]
    )

app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d9cb1ce8272597c6d0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [4]:
import gradio as gr

# State to hold multiple tables
tables = []

# Function to save the table from user inputs
def save_table(name, n1, n2, n3, t1, t2, t3):
    if not name.strip():
        return "❌ Table name is required."

    cols = []
    for n, t in zip([n1, n2, n3], [t1, t2, t3]):
        if n.strip() and t:
            cols.append({"name": n.strip(), "type": t})

    if not cols:
        return "❌ Add at least one valid column."

    tables.append({
        "name": name.strip(),
        "columns": cols
    })

    return f"✅ Table '{name}' saved with {len(cols)} columns."

# Function to generate the CREATE TABLE SQL statements from saved tables
def generate_sql_context():
    if not tables:
        return "⚠️ No tables defined."

    stmts = []
    for table in tables:
        name = table["name"]
        cols = [f"{col['name']} {col['type']}" for col in table["columns"]]
        stmts.append(f"CREATE TABLE {name} (\n  " + ",\n  ".join(cols) + "\n);")

    return "\n\n".join(stmts)

# Function to call the language model and generate SQL
def generate_sql(context, question):
    prompt = f"""### Instruction:
Generate SQL query based on the following context and question.

### Context:
{context}

### Question:
{question}

### Response:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("### Response:")[-1].strip()

# Function to clear everything
def clear_all():
    tables.clear()  # reset global state
    return "", "", "", "", None, None, None, "", "", ""

# Build the Gradio UI
with gr.Blocks(theme="default") as app:
    gr.Markdown("""
    # 🧠 SQL Table & Query Generator
    Easily define SQL tables, generate schemas, and ask questions to get SQL queries.
    """)

    with gr.Accordion("📋 Define Table Schema", open=True):
        with gr.Row():
            table_name = gr.Textbox(label="Table Name", placeholder="e.g. users")

        with gr.Row():
            col_name1 = gr.Textbox(label="Column 1 Name")
            col_type1 = gr.Dropdown(["INT", "VARCHAR", "TEXT", "DATE", "BOOLEAN", "FLOAT"], label="Type")

        with gr.Row():
            col_name2 = gr.Textbox(label="Column 2 Name")
            col_type2 = gr.Dropdown(["INT", "VARCHAR", "TEXT", "DATE", "BOOLEAN", "FLOAT"], label="Type")

        with gr.Row():
            col_name3 = gr.Textbox(label="Column 3 Name")
            col_type3 = gr.Dropdown(["INT", "VARCHAR", "TEXT", "DATE", "BOOLEAN", "FLOAT"], label="Type")

        with gr.Row():
            save_btn = gr.Button("💾 Save Table", variant="primary")
            clear_btn = gr.Button("🧹 Clear All", variant="secondary")
        status = gr.Textbox(label="Status", interactive=False)

    with gr.Accordion("🛠️ Generate SQL Schema", open=True):
        generate_sql_btn = gr.Button("⚙️ Generate SQL Context", variant="secondary")
        context_output = gr.Code(label="SQL Context", language="sql", interactive=False)

    with gr.Accordion("🔎 Ask SQL Question", open=True):
        question_input = gr.Textbox(label="Question", placeholder="e.g. Show all users older than 30", lines=2)
        ask_sql_btn = gr.Button("🚀 Generate SQL Query", variant="primary")
        final_sql_output = gr.Code(label="Generated SQL", language="sql", interactive=False)

    # Event bindings
    save_btn.click(
        save_table,
        inputs=[table_name, col_name1, col_name2, col_name3, col_type1, col_type2, col_type3],
        outputs=[status]
    )

    generate_sql_btn.click(
        generate_sql_context,
        outputs=[context_output]
    )

    ask_sql_btn.click(
        generate_sql,
        inputs=[context_output, question_input],
        outputs=[final_sql_output]
    )

    clear_btn.click(
        clear_all,
        outputs=[
            table_name, col_name1, col_name2, col_name3,
            col_type1, col_type2, col_type3,
            status, context_output, final_sql_output
        ]
    )

app.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2d23c1807c4584ea33.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
